In [1]:
import os
os.chdir('../..')

In [2]:
import numpy as np
import pandas as pd
from collections import Iterable, defaultdict
import random
from os import listdir
from os.path import isfile, join
import json
import pickle

from itertools import combinations

import matplotlib.pyplot as plt

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

D:\Users\glede\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  This is separate from the ipykernel package so we can avoid doing imports until


# Functions

In [3]:
def is_a_DATGAN(name):
    if any(x in name for x in ['TGAN', 'CTGAN', 'FULL', 'TRANSRED', 'LINEAR', 'NOLINKS', 'PREDICTION']):
        return False
    else:
        return True

def compute_stats(freq_list_orig, freq_list_synth):
    """
    Different statistics computed on the frequency list
    
    """
    freq_list_orig, freq_list_synth = np.array(freq_list_orig), np.array(freq_list_synth)
    corr_mat = np.corrcoef(freq_list_orig, freq_list_synth)
    corr = corr_mat[0, 1]
    if np.isnan(corr): corr = 0.0
    # MAE
    mae = np.absolute(freq_list_orig - freq_list_synth).mean()
    # RMSE
    rmse = np.linalg.norm(freq_list_orig - freq_list_synth) / np.sqrt(len(freq_list_orig))
    # SRMSE
    freq_list_orig_avg = freq_list_orig.mean()
    srmse = rmse / freq_list_orig_avg
    # r-square
    u = np.sum((freq_list_synth - freq_list_orig)**2)
    v = np.sum((freq_list_orig - freq_list_orig_avg)**2)
    r2 = 1.0 - u / v
    stat = {'mae': mae, 'rmse': rmse, 'r2': r2, 'srmse': srmse, 'corr': corr}
    
    return stat

# Get all models and associated files

In [4]:
dataset = 'Chicago_DAG'
n_models = 5
n_data = 5

# Models for testing all DATGANS
models = ['CTGAN', 'TGAN']

for i in ['WGAN', 'SGAN', 'WGGP']:
    for j in ['WI', 'OR', 'WO']:
        for k in ['NO', 'BO', 'OD']:
            models.append('{}_{}_{}'.format(i,j,k))
            
# Models for testing different DAGs
models = ['FULL', 'TRANSRED', 'LINEAR', 'NOLINKS', 'PREDICTION']
            
models.sort()

files_ = {}

for m in models:
    tmp = []
    if is_a_DATGAN(m):
        spl = m.split('_')
        for i in range(n_models):
            for j in range(n_data):
                tmp.append('{}_{}_{:0>2}_{}_{:0>2}.csv'.format(spl[0], spl[1], i+1,  spl[2], j+1))
    else:
        for i in range(n_models):
            for j in range(n_data):
                tmp.append('{}_{:0>2}_{:0>2}.csv'.format(m, i+1, j+1))
    files_[m] = tmp


input_folder = '../synth_data/{}/'.format(dataset)

In [5]:
df_orig = pd.read_csv('../data/' + dataset.split('_')[0] + '/data.csv')

In [6]:
if 'Chicago' in dataset:
    continuous_cols = ['distance', 'age', 'departure_time']
elif 'LPMC' in dataset:
    continuous_cols = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [7]:
bins_cont = {}

for c in continuous_cols:
    #bins_cont[c] = pd.qcut(df_orig[c], q=10, retbins=True)[1]
    bins_cont[c] = pd.cut(df_orig[c], bins=10, retbins=True)[1]
    bins_cont[c][0] = -np.inf
    bins_cont[c][-1] = np.inf
    df_orig[c] = pd.cut(df_orig[c], bins=bins_cont[c])

In [8]:
df_orig.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,"(-inf, 6.971]",2,3,3,detached,6,0,"(29.4, 39.2]",1,4,PTE,"(19.093, 21.48]"
1,drive,2,SHOPPING,"(-inf, 6.971]",3,3,3,detached,7,0,"(49.0, 58.8]",1,5,FTE,"(16.707, 19.093]"
2,drive,2,SHOPPING,"(-inf, 6.971]",1,1,0,detached,3,0,"(78.4, 88.2]",1,3,PTE,"(7.16, 9.547]"
3,drive,2,OTHER,"(-inf, 6.971]",2,2,0,detached,5,1,"(39.2, 49.0]",1,5,FTE,"(11.933, 14.32]"
4,passenger,1,SHOPPING,"(-inf, 6.971]",2,2,1,detached,4,0,"(29.4, 39.2]",0,3,Unemployed,"(9.547, 11.933]"


In [9]:
stats_str = ['mae', 'rmse', 'r2', 'srmse', 'corr']
orig_str = 'random-original'

In [10]:
try:
    os.makedirs('./notebooks/results/{}'.format(dataset))
except:
    pass

# Stats per individual column

In [11]:
filepath = './notebooks/results/{}/'.format(dataset)
filename = 'single_columns.pickle'.format(dataset)

all_stats = {}

try:
    all_stats = pickle.load(open(filepath + filename, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')
    try:
        os.makedirs(filepath)
    except:
        pass

Found previous pickel file, using that


In [12]:
# Go through each model
for i, m in enumerate(models):
    
    if m in all_stats:
        print("Results for model \033[1m{}\033[0m ({}/{}) already exists!".format(m, i+1, len(models)))

    else:
        print("Preparing stats for model \033[1m{}\033[0m ({}/{})".format(m, i+1, len(models)))

        all_stats[m] = {}

        for c in df_orig.columns:
            all_stats[m][c] = {}
            for s in stats_str:
                all_stats[m][c][s] = []

        # Load all dataframes for current model
        dfs = [pd.read_csv(input_folder + f) for f in files_[m]]

        # Go through all dataframes generated for each model
        for df in dfs:

            # Discretize continuous columns
            for c in continuous_cols:
                df[c] = pd.cut(df[c], bins=bins_cont[c])

            # Go through each columns
            for c in df_orig.columns:

                agg_vars = [c]

                real = df_orig.copy()
                real['count'] = 1
                real = real.groupby(agg_vars, observed=True).count()
                real /= len(df_orig)

                synth = df.copy()
                synth['count'] = 1
                synth = synth.groupby(agg_vars, observed=True).count()
                synth /= len(df)

                real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
                real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

                sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

                for s in sts:
                    all_stats[m][c][s].append(sts[s])
                    
        pickle.dump(all_stats, open(filepath + filename, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for model FULL (1/5) already exists!
Results for model LINEAR (2/5) already exists!
Results for model NOLINKS (3/5) already exists!
Results for model PREDICTION (4/5) already exists!
Results for model TRANSRED (5/5) already exists!
FINISHED!


In [13]:
if orig_str not in all_stats:

    stats_orig = {}

    for c in df_orig.columns:
        stats_orig[c] = {}
        for s in stats_str:
            stats_orig[c][s] = []

    for i in range(n_models*n_data):

        train = df_orig.sample(int(len(df_orig) * 0.5))
        train.index = range(len(train))
        test = df_orig[~df_orig.index.isin(train.index)]
        test.index = range(len(test))

        # Go through each columns
        for c in df_orig.columns:

            agg_vars = [c]

            real = train.copy()
            real['count'] = 1
            real = real.groupby(agg_vars, observed=True).count()
            real /= len(df_orig)

            synth = test.copy()
            synth['count'] = 1
            synth = synth.groupby(agg_vars, observed=True).count()
            synth /= len(df)

            real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
            real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

            sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

            for s in sts:
                stats_orig[c][s].append(sts[s])
    
    all_stats[orig_str] = stats_orig
    
    pickle.dump(all_stats, open(filepath + filename, 'wb'))

In [14]:
res = {}

for test in ['all', 'cont', 'cat']:
    
    res[test] = {}
    
    if test == 'all':
        cols = df_orig.columns
    elif test == 'cont':
        cols = continuous_cols
    elif test == 'cat':
        cols = set(df_orig.columns) - set(continuous_cols)

    for s in stats_str:
        res[test][s] = {}

    for m in all_stats.keys():

        for s in stats_str:
            res[test][s][m] = []

            for i in range(n_models*n_data):
                tmp = []

                for c in cols:
                    tmp.append(all_stats[m][c][s][i])

                res[test][s][m].append(np.mean(tmp))

In [15]:
avg = {}

for test in ['all', 'cont', 'cat']:
    
    avg[test] = {}

    for s in stats_str:
        avg[test][s] = {}

        for m in all_stats.keys():
            avg[test][s][m] = {
                'mean': np.mean(res[test][s][m]),
                'std': np.std(res[test][s][m])
            }

In [16]:
for test in ['all', 'cont', 'cat']:
    
    if test == 'all':
        str_ = 'on all columns'
    elif test == 'cont':
        str_ = 'on continuous columns'
    elif test == 'cat':
        str_ = 'on categorical columns'
        
    for s in ['srmse']:#stats:
        print('Ranking {} based on {}:'.format(str_, s.upper()))

        if s in ['r2', 'corr']:
            sorted_dct = {k: v for k, v in sorted(avg[test][s].items(), key=lambda item: item[1]['mean'])[::-1]}
        else:
            sorted_dct = {k: v for k, v in sorted(avg[test][s].items(), key=lambda item: item[1]['mean'])}

        for i, item in enumerate(sorted_dct):
            print('  {:>2}. {:<20} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
        print()


Ranking on all columns based on SRMSE:
   1. random-original      - 5.25e-02 ± 2.85e-03
   2. NOLINKS              - 5.90e-02 ± 5.88e-03
   3. LINEAR               - 6.00e-02 ± 4.29e-03
   4. TRANSRED             - 6.19e-02 ± 6.07e-03
   5. PREDICTION           - 6.25e-02 ± 2.72e-03
   6. FULL                 - 6.49e-02 ± 9.76e-03

Ranking on continuous columns based on SRMSE:
   1. random-original      - 4.29e-02 ± 3.76e-03
   2. FULL                 - 1.26e-01 ± 2.11e-02
   3. LINEAR               - 1.32e-01 ± 1.94e-02
   4. TRANSRED             - 1.35e-01 ± 9.41e-03
   5. NOLINKS              - 1.77e-01 ± 3.07e-02
   6. PREDICTION           - 1.96e-01 ± 1.20e-02

Ranking on categorical columns based on SRMSE:
   1. PREDICTION           - 2.91e-02 ± 2.49e-03
   2. NOLINKS              - 2.94e-02 ± 3.04e-03
   3. LINEAR               - 4.20e-02 ± 3.56e-03
   4. TRANSRED             - 4.36e-02 ± 5.83e-03
   5. FULL                 - 4.95e-02 ± 7.94e-03
   6. random-original      - 5.49

# Stats per couple columns

In [17]:
combs = []

for k in combinations(df_orig.columns, 2):
    combs.append(k[0] + '::' + k[1])

In [18]:
filepath = './notebooks/results/{}/'.format(dataset)
filename = 'couple_combinations.pickle'.format(dataset)

all_stats = {}

try:
    all_stats = pickle.load(open(filepath + filename, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')
    try:
        os.makedirs(filepath)
    except:
        pass

Found previous pickel file, using that


In [19]:
# Go through each model
for i, m in enumerate(models):
    
    if m in all_stats:
        print("Results for model \033[1m{}\033[0m ({}/{}) already exists!".format(m, i+1, len(models)))

    else:
        print("Preparing stats for model \033[1m{}\033[0m ({}/{})".format(m, i+1, len(models)))

        all_stats[m] = {}

        for c in combs:
            all_stats[m][c] = {}
            for s in stats_str:
                all_stats[m][c][s] = []

        # Load all dataframes for current model
        dfs = [pd.read_csv(input_folder + f) for f in files_[m]]

        # Go through all dataframes generated for each model
        for df in dfs:

            # Discretize continuous columns
            for c in continuous_cols:
                df[c] = pd.cut(df[c], bins=bins_cont[c])

            # Go through each columns
            for c in combs:

                agg_vars = c.split('::')

                real = df_orig.copy()
                real['count'] = 1
                real = real.groupby(agg_vars, observed=True).count()
                real /= len(df_orig)

                synth = df.copy()
                synth['count'] = 1
                synth = synth.groupby(agg_vars, observed=True).count()
                synth /= len(df)

                real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
                real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

                sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

                for s in sts:
                    all_stats[m][c][s].append(sts[s])
                    
        pickle.dump(all_stats, open(filepath + filename, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for model FULL (1/5) already exists!
Results for model LINEAR (2/5) already exists!
Results for model NOLINKS (3/5) already exists!
Results for model PREDICTION (4/5) already exists!
Results for model TRANSRED (5/5) already exists!
FINISHED!


In [20]:
if orig_str not in all_stats:
    stats_orig = {}

    for c in combs:
        stats_orig[c] = {}
        for s in stats_str:
            stats_orig[c][s] = []

    for i in range(n_models*n_data):

        train = df_orig.sample(int(len(df_orig) * 0.5))
        train.index = range(len(train))
        test = df_orig[~df_orig.index.isin(train.index)]
        test.index = range(len(test))

        # Go through each columns
        for c in combs:

            agg_vars = c.split('::')

            real = train.copy()
            real['count'] = 1
            real = real.groupby(agg_vars, observed=True).count()
            real /= len(df_orig)

            synth = test.copy()
            synth['count'] = 1
            synth = synth.groupby(agg_vars, observed=True).count()
            synth /= len(df)

            real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
            real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

            sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

            for s in sts:
                stats_orig[c][s].append(sts[s])
                
    all_stats[orig_str] = stats_orig
    
    pickle.dump(all_stats, open(filepath + filename, 'wb'))

In [21]:
res = {}

for s in stats_str:
    res[s] = {}

for m in all_stats.keys():

    for s in stats_str:
        res[s][m] = []

        for i in range(n_models*n_data):
            tmp = []

            for c in combs:
                tmp.append(all_stats[m][c][s][i])

            res[s][m].append(np.mean(tmp))

In [22]:
avg = {}

for s in stats_str:
    avg[s] = {}

    for m in all_stats.keys():
        avg[s][m] = {
            'mean': np.mean(res[s][m]),
            'std': np.std(res[s][m])
        }

In [23]:
for s in ['srmse']:#stats:
    print('Ranking on all coupled combinations based on {}:'.format(s.upper()))

    if s in ['r2', 'corr']:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])[::-1]}
    else:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])}

    for i, item in enumerate(sorted_dct):
        print('  {:>2}. {:<20} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
    print()


Ranking on all coupled combinations based on SRMSE:
   1. random-original      - 1.28e-01 ± 4.67e-03
   2. TRANSRED             - 1.81e-01 ± 1.04e-02
   3. LINEAR               - 1.84e-01 ± 9.46e-03
   4. FULL                 - 1.85e-01 ± 1.40e-02
   5. NOLINKS              - 3.48e-01 ± 1.14e-02
   6. PREDICTION           - 3.48e-01 ± 4.71e-03



# Stats per trouple columns

In [24]:
combs = []

for k in combinations(df_orig.columns, 3):
    combs.append(k[0] + '::' + k[1] + '::' + k[2])

In [25]:
filepath = './notebooks/results/{}/'.format(dataset)
filename = 'trouple_combinations.pickle'.format(dataset)

all_stats = {}

try:
    all_stats = pickle.load(open(filepath + filename, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')
    try:
        os.makedirs(filepath)
    except:
        pass

Found previous pickel file, using that


In [26]:
# Go through each model
for i, m in enumerate(models):

    if m in all_stats:
        print("Results for model \033[1m{}\033[0m ({}/{}) already exists!".format(m, i+1, len(models)))
    else:
        print("Preparing stats for model \033[1m{}\033[0m ({}/{})".format(m, i+1, len(models)))

        all_stats[m] = {}

        for c in combs:
            all_stats[m][c] = {}
            for s in stats_str:
                all_stats[m][c][s] = []

        # Load all dataframes for current model
        dfs = [pd.read_csv(input_folder + f) for f in files_[m]]

        # Go through all dataframes generated for each model
        for df in dfs:

            # Discretize continuous columns
            for c in continuous_cols:
                df[c] = pd.cut(df[c], bins=bins_cont[c])

            # Go through each columns
            for c in combs:

                agg_vars = c.split('::')

                real = df_orig.copy()
                real['count'] = 1
                real = real.groupby(agg_vars, observed=True).count()
                real /= len(df_orig)

                synth = df.copy()
                synth['count'] = 1
                synth = synth.groupby(agg_vars, observed=True).count()
                synth /= len(df)

                real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
                real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

                sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

                for s in sts:
                    all_stats[m][c][s].append(sts[s])
    
        pickle.dump(all_stats, open(filepath + filename, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for model FULL (1/5) already exists!
Results for model LINEAR (2/5) already exists!
Results for model NOLINKS (3/5) already exists!
Results for model PREDICTION (4/5) already exists!
Results for model TRANSRED (5/5) already exists!
FINISHED!


In [27]:
if orig_str not in all_stats:
    stats_orig = {}

    for c in combs:
        stats_orig[c] = {}
        for s in stats_str:
            stats_orig[c][s] = []

    for i in range(n_models*n_data):

        train = df_orig.sample(int(len(df_orig) * 0.5))
        train.index = range(len(train))
        test = df_orig[~df_orig.index.isin(train.index)]
        test.index = range(len(test))

        # Go through each columns
        for c in combs:

            agg_vars = c.split('::')

            real = train.copy()
            real['count'] = 1
            real = real.groupby(agg_vars, observed=True).count()
            real /= len(df_orig)

            synth = test.copy()
            synth['count'] = 1
            synth = synth.groupby(agg_vars, observed=True).count()
            synth /= len(df)

            real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
            real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

            sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

            for s in sts:
                stats_orig[c][s].append(sts[s])
                
    all_stats[orig_str] = stats_orig
    
    pickle.dump(all_stats, open(filepath + filename, 'wb'))

In [28]:
res = {}

for s in stats_str:
    res[s] = {}

for m in all_stats.keys():

    for s in stats_str:
        res[s][m] = []

        for i in range(n_models*n_data):
            tmp = []

            for c in combs:
                tmp.append(all_stats[m][c][s][i])

            res[s][m].append(np.mean(tmp))

In [29]:
avg = {}

for s in stats_str:
    avg[s] = {}

    for m in all_stats.keys():
        avg[s][m] = {
            'mean': np.mean(res[s][m]),
            'std': np.std(res[s][m])
        }

In [30]:
for s in ['srmse']:#stats_str:
    print('Ranking on all triple combinations based on {}:'.format(s.upper()))

    if s in ['r2', 'corr']:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])[::-1]}
    else:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])}

    for i, item in enumerate(sorted_dct):
        print('  {:>2}. {:<20} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
    print()


Ranking on all triple combinations based on SRMSE:
   1. random-original      - 2.40e-01 ± 9.08e-03
   2. TRANSRED             - 3.76e-01 ± 1.45e-02
   3. FULL                 - 3.80e-01 ± 1.84e-02
   4. LINEAR               - 3.90e-01 ± 1.56e-02
   5. PREDICTION           - 7.85e-01 ± 8.43e-03
   6. NOLINKS              - 7.91e-01 ± 1.42e-02

